# Projeto 15: Geração de imagens com GANs

- Código adaptado da documentação: https://www.tensorflow.org/beta/tutorials/generative/dcgan




# Etapa 1: Importação das bibliotecas

In [68]:
#%tensorflow_version 2.x
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display
import tensorflow as tf
tf.__version__

'2.5.0'

# Etapa 2: Importação e tratamento da base de dados

In [69]:
(train_images, train_labels), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

In [70]:
train_images.shape

(60000, 28, 28)

In [71]:
train_labels.shape

(60000,)

In [72]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

In [73]:
train_images.shape

(60000, 28, 28, 1)

In [74]:
# Normalização entre -1 e 1
train_images = (train_images - 127.5) / 127.5

In [75]:
buffer_size = 60000
batch_size = 256

In [76]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)

In [77]:
train_dataset

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>

# Etapa 3: Visualização dos dados

In [78]:
plt.imshow(train_images[112].reshape((28,28)) , cmap = 'gray')

# Etapa 4: Construção do gerador

In [79]:
def make_generator_model():
  model = tf.keras.Sequential()
  model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,))) # 12544
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Reshape((7, 7, 256)))

  # 7x7x128
  model.add(layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  # 14x14x64
  model.add(layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  # 28x28x1
  model.add(layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh'))

  model.summary()

  return model

In [80]:
noise = tf.random.normal([1, 100])

In [81]:
noise

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[ 0.7159807 ,  0.28294134, -0.25798827,  0.72260183, -0.534275  ,
         0.6763111 , -1.154376  , -1.1432737 , -1.2389333 ,  0.24561274,
         0.7089412 ,  0.3698601 , -0.05839563, -0.8281396 , -0.90386176,
         0.05175862,  0.32874402,  0.67180526,  0.7941531 , -1.0044975 ,
         1.5055916 , -0.6380733 , -0.17803155,  0.18447876, -1.19522   ,
        -2.8498838 , -1.0560544 , -0.0620461 , -0.8413869 ,  0.09913646,
         0.8832447 , -0.55523694,  0.6007203 ,  0.66744787, -1.0768542 ,
        -0.06257309,  0.9052609 , -1.4694582 ,  1.7874004 ,  0.10168468,
         0.9087417 ,  1.0710814 ,  1.1021453 , -1.7224982 ,  0.53424454,
        -0.594372  , -0.8852036 ,  1.4051229 , -0.23824628,  0.49048012,
        -0.37184653, -1.2206283 ,  1.1908646 , -0.7800827 , -2.3074963 ,
        -1.3385727 ,  1.1384937 ,  0.57656133,  1.6824932 ,  0.26960248,
        -0.5634068 ,  1.1233565 ,  0.6061422 , -1.7660315 , -0.5447448 ,
 

In [82]:
generator = make_generator_model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 12544)             1254400   
_________________________________________________________________
batch_normalization_6 (Batch (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_7 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 7, 7, 128)        

In [83]:
generated_image = generator(noise, training = False)

In [84]:
generated_image.shape

TensorShape([1, 28, 28, 1])

In [85]:
plt.imshow(generated_image[0,:,:,0], cmap='gray')

# Etapa 5: Construção do discriminador

In [86]:
def make_discrimator_model():
  model = tf.keras.Sequential()
  # 14x14x64
  model.add(layers.Conv2D(64, (5,5), strides = (2,2), padding='same', input_shape=[28,28,1]))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  # 7x7x128
  model.add(layers.Conv2D(128, (5,5), strides = (2,2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  # 6272
  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  model.summary()

  return model

In [87]:
discriminator = make_discrimator_model()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)             

In [88]:
decision = discriminator(generated_image, training = False)
decision

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00013059]], dtype=float32)>

# Etapa 6: Loss function para as duas redes

In [89]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [90]:
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

In [91]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [92]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4) # 0.00001
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4) # 0.00001

In [93]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Etapa 7: Treinamento do modelo 

In [94]:
epochs = 100
noise_dim = 100
num_examples_to_generate = 16

In [95]:
seed = tf.random.normal([num_examples_to_generate, noise_dim])
seed.shape

TensorShape([16, 100])

In [96]:
batch_size

256

In [97]:
@tf.function
def train_steps(images):
  noise = tf.random.normal([batch_size, noise_dim])
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    # Chamar o gerador e passar os números aleatórios
    generated_images = generator(noise, training = True)

    # Passar para o discriminador as imagens reais e falsas para fazer a classificação
    real_output = discriminator(images, training = True)
    fake_output = discriminator(generated_images, training = True)

    # Calcular a loss para o gerador e discriminador
    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  # Calcular o gradiente das losses para atualizar os pesos
  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  # Aplicar os otimizadores e atualizar os pesos
  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [98]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_steps(image_batch)

    display.clear_output(wait = True)
    generate_and_save_images(generator, epoch + 1, seed)

    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix=checkpoint_prefix)

    print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))

  display.clear_output(wait = True)
  generate_and_save_images(generator, epoch + 1, seed) 

In [99]:
image_dir = './training_images'

In [100]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training = False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i + 1)
    plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    plt.axis('off')
  plt.savefig('{}/image_at_epoch_{:04d}.png'.format(image_dir, epoch))
  plt.show()

In [101]:
train(train_dataset, epochs)

FileNotFoundError: [Errno 2] No such file or directory: './training_images\\ckptimage_at_epoch_0001.png'

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('{}/image_at_epoch_{:04d}.png'.format(image_dir, epoch_no))

In [ ]:
display_image(60)